In [1]:
import os
import pandas as pd
import pathway as pw

In [2]:
pw.set_license_key('F0EEC7-FA3FF6-216017-196BDE-2DAC9A-V3')
os.environ["GROQ_API_KEY"] = "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"

import logging
logging.basicConfig(level=logging.CRITICAL)

In [3]:
import nltk
print(nltk.__version__)

3.9.1


In [4]:
# # Run Once
# import nltk
# nltk.download('punkt', download_dir='/root/nltk_data')
# nltk.download('punkt_tab')

os.environ["RUST_BACKTRACE"] = "full"

In [5]:
""" Get Docs, Parser, Splitter"""

folder = pw.io.fs.read(
    path="./data/",
    format="binary",
    with_metadata=True,
)

sources = [
    folder,
]


from pathway.xpacks.llm import llms, parsers, prompts
# chat = llms.OpenAIChat(model="gpt-4o")
# table_args = {
#    "parsing_algorithm": "llm",
#    "llm": chat,
#    "prompt": prompts.DEFAULT_MD_TABLE_PARSE_PROMPT,
# }
# image_args = {
#     "parsing_algorithm": "llm",
#     "llm": chat,
#     "prompt": prompts.DEFAULT_IMAGE_PARSE_PROMPT,
# }
# parser = parsers.OpenParse(
#     table_args=table_args, 
#     image_args=image_args,
#     processing_pipeline=None # defaults to CustomIngestionPipeline() defined in _openparse_utils, we can change this by defining our own pipeline from openparse
# )
parser = parsers.ParseUnstructured()


from pathway.xpacks.llm.splitters import TokenCountSplitter
# splitter = None # OpenParse handles the text splitting
splitter = TokenCountSplitter()



from pathway.stdlib.indexing.data_index import DataIndex, InnerIndex
from pathway.stdlib.indexing.retrievers import AbstractRetrieverFactory,InnerIndexFactory
from pathway.stdlib.indexing.vector_document_index import default_usearch_knn_document_index
from pathway.xpacks.llm._utils import _coerce_sync

from pathway.xpacks.llm import embedders
embedder = embedders.SentenceTransformerEmbedder(
    model="intfloat/e5-large-v2",
    call_kwargs={}, # optional additional parameters to give to embedder
    device="cpu",
)
#embedder = embedders.OpenAIEmbedder(cache_strategy=DiskCache())

class CustomRetrieverFactor(InnerIndexFactory):
    def build_inner_index():

        return
    def build_index(
        self,
        data_column: pw.ColumnReference,
        data_table: pw.Table,
        metadata_column: pw.ColumnExpression | None = None,
    ) -> DataIndex:
        
        self.embedder = embedder
        self.embedding_dimension = len(_coerce_sync(self.embedder.__wrapped__)("."))
        knn_index = default_usearch_knn_document_index(
            data_column, #chunked_docs.text,
            data_table, #chunked_docs,
            dimensions=self.embedding_dimension,
            metadata_column=metadata_column, #metadata_column=chunked_docs.data["metadata"],
            embedder=self.embedder,
        )
        # inner_index = build_inner_index()
        # knn_index.inner_index 
        return knn_index

retriever_factory = CustomRetrieverFactor()

from pathway.xpacks.llm.document_store import DocumentStore
document_store = DocumentStore(
    *sources,
    retriever_factory=retriever_factory, 
    parser=parser, 
    splitter=splitter,
)

/code/Pathway_RAG/env1/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
# """ IO Test """
# !printf "id,owner,pet\\n1,Alice,dog\\n2,Bob,dog\\n3,Alice,cat\\n4,Bob,dog" > dataset.csv

# import pathway as pw
# class InputSchema(pw.Schema):
#   owner: str
#   pet: str
# t = pw.io.fs.read("dataset.csv", format="csv", schema=InputSchema)
# pw.debug.compute_and_print(t, include_id=False)

In [7]:
""" Make Document Store Servers """

RUST_BACKTRACE=1
host = "127.0.0.1" # define server address
port = 8667 # define server port

In [8]:
from pathway.xpacks.llm.servers import DocumentStoreServer
server = DocumentStoreServer(host=host,port=port,document_store=document_store) # initialize document store server

server.run(threaded=True, with_cache=True, cache_backend=pw.persistence.Backend.filesystem("./Cache")) # start server

/code/Pathway_RAG/env1/lib/python3.12/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Sequence[str] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


<Thread(Thread-6 (run), started 140336489133760)>

In [9]:
metadata = pd.read_csv('metadata_filename_parties.csv')
metadata

,Filename,Cleaned_Parties
0,CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605...,"['Birch First Global Investments Inc.', 'Mount..."
1,EuromediaHoldingsCorp_20070215_10SB12G_EX-10.B...,"['Rogers Cable Communications Inc.', 'EuroMedi..."
2,FulucaiProductionsLtd_20131223_10-Q_EX-10.9_83...,"['CONVERGTV,\xa0INC.', 'Fulucai Productions Lt..."
3,GopageCorp_20140221_10-K_EX-10.1_8432966_EX-10...,"['PSiTech Corporation', 'Empirical Ventures, I..."
4,IdeanomicsInc_20160330_10-K_EX-10.26_9512211_E...,['Beijing Sun Seven Stars Culture Development ...
...,...,...
505,NEONSYSTEMSINC_03_01_1999-EX-10.5-DISTRIBUTOR ...,"['Peregrine/Bridge Transfer Corporation', 'Neo..."
506,WOMENSGOLFUNLIMITEDINC_03_29_2000-EX-10.13-END...,"['SQUARE TWO GOLF INC.', 'KATHY WHITWORTH']"
507,WOMENSGOLFUNLIMITEDINC_03_29_2000-EX-10.13-END...,"['SQUARE TWO GOLF INC.', 'KATHY WHITWORTH']"
508,"KUBIENT,INC_07_02_2020-EX-10.14-MASTER SERVICE...","['Kubient, Inc.', 'The Associated Press']"


In [10]:
type(metadata['Filename'].iloc[0])

str

In [11]:
import re
def extract_companies_from_question(question):
    pattern = r'between\s+(.+?)\s+and\s+(.+?)\?'
    
    match = re.search(pattern, question)
    
    if match:
        # Extract the two companies
        company_1 = match.group(1).strip()
        company_2 = match.group(2).strip()
        return [company_1, company_2]
    else:
        return None

In [12]:
def find_contract_filename(companies, company_contract_df):
    if len(companies) != 2:
        return "Error: Two companies are needed."

    # Normalize company names for case-insensitive matching (strip spaces and lower case)
    company1, company2 = [company.strip().lower() for company in companies]
    print(company1, company2)

    # Loop through each contract to check if the companies are involved
    for _, row in company_contract_df.iterrows():
        # Clean and normalize the party list in the row
        cleaned_parties = eval(row['Cleaned_Parties'])  # Convert string list to actual list
        cleaned_parties = [party.strip().lower() for party in cleaned_parties]

        # Check if both companies are in the list of parties
        if company1 in cleaned_parties and company2 in cleaned_parties:
            return row['Filename']  # Return the corresponding contract filename
    
    return "No contract found between the given companies."

# Example CSV file path
csv_file_path = 'metadata_filename_parties.csv'
company_contract_df = pd.read_csv(csv_file_path)

query = "What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?"

companies = extract_companies_from_question(query)
if companies:
    print(f"The two companies are: {companies}")
    contract_filename = find_contract_filename(companies, company_contract_df)
    print(contract_filename.split(".pdf"))
    l = contract_filename.split(".pdf")
    l.append('.txt')
    contract_filename = ''.join(l)
    print(''.join(l))
    print(f"The contract filename is: {contract_filename}")
else:
    print("No companies found in the query.")


The two companies are: ['Birch First Global Investments Inc.', 'Mount Kowledge Holdings Inc.']
birch first global investments inc. mount kowledge holdings inc.
['CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement', '']
CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.txt
The contract filename is: CybergyHoldingsInc_20140520_10-Q_EX-10.27_8605784_EX-10.27_Affiliate Agreement.txt


In [13]:
companies = extract_companies_from_question(query)
companies

['Birch First Global Investments Inc.', 'Mount Kowledge Holdings Inc.']

In [16]:
""" Make DocumentStoreClient """
import json
import requests

class DocumentStoreClient:
    """
    A client you can use to query DocumentStoreServer.

    Please provide either the `url`, or `host` and `port`.

    Args:
        host: host on which `DocumentStoreServer </developers/api-docs/pathway-xpacks-llm/documentstore#pathway.xpacks.llm.document_store.DocumentStoreServer>`_ listens
        port: port on which `DocumentStoreServer </developers/api-docs/pathway-xpacks-llm/documentstore#pathway.xpacks.llm.document_store.DocumentStoreServer>`_ listens
        url: url at which `DocumentStoreServer </developers/api-docs/pathway-xpacks-llm/documentstore#pathway.xpacks.llm.document_store.DocumentStoreServer>`_ listens
        timeout: timeout for the post requests in seconds
    """  # noqa

    def __init__(
        self,
        host: str | None = None,
        port: int | None = None,
        url: str | None = None,
        timeout: int | None = 300,
        additional_headers: dict | None = None,
    ):
        err = "Either (`host` and `port`) or `url` must be provided, but not both."
        if url is not None:
            if host or port:
                raise ValueError(err)
            self.url = url
        else:
            if host is None:
                raise ValueError(err)
            port = port or 80
            self.url = f"http://{host}:{port}"

        self.timeout = timeout
        self.additional_headers = additional_headers or {}

    def query(
        self,
        query: str,
        k: int = 3,
        metadata_filter: str | None = None,
        filepath_globpattern: str | None = None,
    ) -> list[dict]:
        """
        Perform a query to the vector store and fetch results.

        Args:
            query:
            k: number of documents to be returned
            metadata_filter: optional string representing the metadata filtering query
                in the JMESPath format. The search will happen only for documents
                satisfying this filtering.
            filepath_globpattern: optional glob pattern specifying which documents
                will be searched for this query.
        """

        data = {"query": query, "k": k}
        print("lol")
        print(data)

        query = data['query']
        print(query)
        # companies = extract_companies_from_question(query)
        companies = True
        # print(companies)

        if companies:
            # metadata_filter = find_contract_filename(companies, company_contract_df)
            # metadata_filter = metadata_filter.split(".pdf")[0].replace('-', '_').replace('.', '_').replace(' ','_')
            # print(metadata_filter)
            
            if metadata_filter is not None:
                data["metadata_filter"] = metadata_filter#f"""contains(path, '{metadata_filter}')"""
            if filepath_globpattern is not None:
                data["filepath_globpattern"] = filepath_globpattern
            url = self.url + "/v1/retrieve"
            print(data)
            response = requests.post(
                url,
                data=json.dumps(data),
                headers=self._get_request_headers(),
                timeout=self.timeout,
            )

            responses = response.json()
            return sorted(responses, key=lambda x: x["dist"])
        
        else:
            print("Weren't able to trace company")

    # Make an alias
    __call__ = query

    def _get_request_headers(self):
        request_headers = {"Content-Type": "application/json"}
        request_headers.update(self.additional_headers)
        return request_headers

In [ ]:
""" Showcasing Retriver Functionality """
client = DocumentStoreClient(
    host=host,
    port=port,
)

queryy = "What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?"
results = client.query(queryy, filepath_globpattern="**/CybergyHoldingsInc_20140520_10_Q_EX_10_27_8605784_EX_10_27_Affiliate_Agreement.txt")

lol
{'query': 'What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?', 'k': 3}
What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?
{'query': 'What is the Agreement Date for the contract between Birch First Global Investments Inc. and Mount Kowledge Holdings Inc.?', 'k': 3, 'filepath_globpattern': '**/CybergyHoldingsInc_20140520_10_Q_EX_10_27_8605784_EX_10_27_Affiliate_Agreement.txt'}


In [ ]:
##########################

In [28]:
context = []
for result in results:
    context.append(result["text"])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

# retriever = client.as_retriever()

template = """
You are smart assistant that helps users with their documents on Google Drive and Sharepoint.
Given a context, respond to the user question.
CONTEXT:
{context}
QUESTION: {question}
YOUR ANSWER:"""

prompt = ChatPromptTemplate.from_template(template)
# llm = ChatOpenAI()
model = ChatGroq(temperature=0, model_name="llama3-70b-8192")
chain_summarize = prompt | model
response = chain_summarize.invoke({"context" : context, "question" : queryy})
print(response)

In [ ]:
chain.invoke("What is Pathway?")

In [ ]:
print(client.get_vectorstore_statistics())
print(client.get_input_files())

In [ ]:
import pandas as pd 
df = pd.read_csv('contract_questions_and_answers')


In [ ]:
queries = df['Question'].tolist()
inference_results = []

for query in queries:
    print(f"Querying: {query}")
    response = client.query(query, k=3)  
    inference_results.append({
        "query": query,
        "response": response  
    })

In [ ]:
def evaluate_answer(predicted, actual):
    try:
        predicted_num = float(predicted)
        actual_num = float(actual)
        absolute_diff = abs(predicted_num - actual_num)
        relative_diff = abs((predicted_num - actual_num) / actual_num) * 100 if actual_num != 0 else float('inf')
        return {
            'type': 'numeric',
            'absolute_diff': absolute_diff,
            'relative_diff_percentage': relative_diff
        }
    except ValueError:
        predicted_str = str(predicted).strip().lower()
        actual_str = str(actual).strip().lower()
        exact_match = predicted_str == actual_str
        similarity = SequenceMatcher(None, predicted_str, actual_str).ratio() * 100  
        
        return {
            'type': 'string',
            'exact_match': exact_match,
            'similarity_percentage': similarity
        }

In [ ]:
df["response"] = inference_results

eval_results = []
for index, row in df.iterrows():
    predicted = row['response'] 
    actual = row['Answer']  
    evaluation = evaluate_answer(predicted, actual)
    eval_results.append(evaluation)

df['eval'] = eval_results
df.to_csv('results.csv', index=False)